<a href="https://colab.research.google.com/github/amityu/BigData/blob/main/reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 31.9 MB/s eta 0:00:00


In [7]:
import pandas as pd
from pytorch_lightning.utilities.types import EVAL_DATALOADERS
#import YARCN
import torch
import torch.nn as nn
import random
import numpy as np
import torchmetrics
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl

from torch.utils.data import random_split
CHECK_PATH =r'D:\amityu\rec_data' +'/cp/'
dnn_path = 'C:/Users/amityu/red_data_c/dnn/'
DATA_PATH = 'C:/Users/amityu/red_data_c/'
DATA_PATHD = 'D:/amityu/rec_data/'

class RCM(pl.LightningModule):

    def __init__(self,max_user_id= 544873, max_item_id=34500, max_source=50000, max_campaign = 14500, dim=64,
                 user_id_dim = 1, item_id_dim=8, source_dim =8, campaign_dim = 8,
                 placement_id_dim = 8, syndicator_dim = 8, region_dim =8,
                 placement_max =1141, syndicator_max = 2011, region_max = 891):
        super().__init__()

        self.save_hyperparameters()
        self.placement_id = nn.Embedding(num_embeddings=placement_max, embedding_dim=placement_id_dim)
        self.syndicator = nn.Embedding(num_embeddings=syndicator_max, embedding_dim=syndicator_dim)
        self.region = nn.Embedding(num_embeddings=region_max, embedding_dim=region_dim)
        self.target_id = nn.Embedding(num_embeddings=max_item_id, embedding_dim=item_id_dim)

        #self.user_id = nn.Embedding(num_embeddings=max_user_id, embedding_dim=user_id_dim)
        self.source = nn.Embedding(num_embeddings=max_source, embedding_dim=source_dim)

        self.campaign = nn.Embedding(num_embeddings=max_campaign, embedding_dim=campaign_dim)

        self.fc_user = nn.Linear(in_features=len(user_numerical_columns + user_oh_cat_columns), out_features=dim)
        #self.fc_user_full = nn.Linear(in_features=user_id_dim + region_dim + dim, out_features=dim)
        self.fc_user_full = nn.Linear(in_features= region_dim + dim, out_features=dim)
        #self.fc_user_full = nn.Linear(in_features=dim, out_features=dim)

        self.fc_item = nn.Linear(in_features=len(item_oh_cat_columns + target_numerical_columns) , out_features=dim)
        self.fc_item_full = nn.Linear(in_features= dim + campaign_dim + source_dim + item_id_dim + syndicator_dim +
                                                   placement_id_dim, out_features=dim)
        #self.fm = nn.Linear(in_features = dim*2, out_features =dim)
        self.deep = nn.Linear(in_features=dim*2, out_features=dim*2)
        self.validation_auc = torchmetrics.AUROC(task='binary')
        self.output = nn.Linear(2*dim, out_features=1)
        self.dim = dim

    def forward(self, x):
        (user, user_id,region, item, target_id, source,campaign, syndicator, placement_id) = x
        # Pass through embedding layers
        relu = nn.ReLU()
        user_embedded = relu(self.fc_user(user))
        #user_id_embedded = self.user_id(user_id)
        region_embedded = self.region(region)
        #user_embedded_full = relu(self.fc_user_full(torch.cat([user_id_embedded, region_embedded, user_embedded], dim=1)))
        user_embedded_full = relu(self.fc_user_full(torch.cat([region_embedded, user_embedded], dim=1)))


        item_embedded = relu(self.fc_item(item))
        item_id_embedded = self.target_id(target_id)
        source_embedded = self.source(source)
        campaign_embedded = self.campaign(campaign)
        syndicator_embedded = self.syndicator(syndicator)
        placement__embedded = self.placement_id(placement_id)
        item_embedded_full = relu(self.fc_item_full(torch.cat([item_embedded, item_id_embedded, source_embedded, campaign_embedded, syndicator_embedded, placement__embedded], dim=1)))

        #fac_machine_vector = relu(self.fm(torch.cat([user_embedded_full, item_embedded_full], dim=1)))
        deep_vector = relu(self.deep(torch.cat([user_embedded_full, item_embedded_full], dim=1)))
        pred = self.output(deep_vector)
        return pred

    def training_step(self, batch, batch_idx):
        x, labels = batch

        predicted_labels = self(x)
        loss = nn.BCEWithLogitsLoss()(predicted_labels.view(-1,1), labels.view(-1,1).float())
        return loss

    def validation_step(self, batch, batch_idx):
        #(user, user_id,region, item, target_id, source,campaign, syndicator, placement_id), labels = batch
        x, labels = batch

        predicted_logits = self(x)
        loss = nn.BCEWithLogitsLoss()(predicted_logits.view(-1,1), labels.view(-1,1).float())
        self.log('val_loss', loss, prog_bar=True)
        preds = torch.sigmoid(predicted_logits)
        self.validation_auc.update(preds.squeeze(), labels)
        self.log('val_auc', self.validation_auc, prog_bar=True)

    #def on_validation_epoch_end(self, outputs):
    #    self.log('val_auc_epoch', self.validation_auc.compute(), prog_bar=True)
    #    self.validation_auc.reset()
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr = 0.01)
        return opt




In [3]:

class FMds2(Dataset):
    def __init__(self,ds_df):
        ds_df = ds_df.astype(np.float32)
        self.user = ds_df[user_oh_cat_columns + user_numerical_columns]
        self.item = ds_df[item_oh_cat_columns + target_numerical_columns]
        self.target_id = ds_df['target_id_hash_code'].astype(int)
        self.source = ds_df['source_id_hash_code'].astype(int)
        self.campaign = ds_df['campaign_id_hash_code'].astype(int)
        self.user_id = ds_df['user_id_hash_code'].astype(int)
        self.region = ds_df['region_code'].astype(int)
        self.syndicator = ds_df['syndicator_code'].astype(int)
        self.placement_id = ds_df['placement_id_code'].astype(int)
        self.labels = ds_df['is_click'].astype(int)
    def __getitem__(self, index):
        return (torch.tensor(self.user.iloc[index]), torch.tensor(self.user_id.iloc[index]), torch.tensor(self.region.iloc[index]),
                torch.tensor(self.item.iloc[index]), torch.tensor(self.target_id.iloc[index]), torch.tensor(self.source.iloc[index]),
                torch.tensor(self.campaign.iloc[index]), torch.tensor(self.syndicator.iloc[index]), torch.tensor(self.placement_id.iloc[index]),
                ), torch.tensor(self.labels.iloc[index])
    def __len__(self):
        return len (self.user)




In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/reco2/train_small_few_cat.csv')


In [ ]:
user_categorial_columns = [
        'browser_platform',
        'os_family',
        'country_code',
        'region',
        'day_of_week',
        'time_of_day',
        'gmt_offset']
user_numerical_columns = ['page_view_start_time','user_recs', 'user_clicks',    'user_target_recs']

target_categorical_columns = ['syndicator_id_hash', 'target_item_taxonomy', 'placement_id_hash', 'publisher_id_hash',  'source_item_type']

target_embedding = ['campaign_id_hash', 'source_id_hash', 'target_id_hash']
target_numerical_columns = ['empiric_calibrated_recs',    'empiric_clicks']
#%%
# Assume df is your DataFrame and user_categorical_columns is your list of categorical columns

user_oh_cat_columns = []
user_oh_indices = []
for cat_col in user_categorial_columns:
    # Get columns that start with cat_col and their indices
    user_oh_cat_columns += [col  for col in df.columns if col.startswith(cat_col)]
    user_oh_indices += [i for (i,col) in enumerate(df.columns) if col.startswith(cat_col)]

item_oh_cat_columns = []
item_oh_indices = []
for cat_col in target_categorical_columns:
    item_oh_cat_columns += [col  for col in df.columns if col.startswith(cat_col)]
    item_oh_indices += [i for (i,col) in enumerate(df.columns) if col.startswith(cat_col)]

user_num_indices = []

for num_col in user_numerical_columns:
    user_num_indices += [i for (i,col) in enumerate(df.columns) if col.startswith(num_col)]

item_num_indices = []
for num_col in target_numerical_columns:
    item_num_indices += [i for (i,col) in enumerate(df.columns) if col.startswith(num_col)]

campaign_index = []
for col in target_embedding:
    campaign_index += [i for (i,col) in enumerate(df.columns) if col== 'campaign_id_hash_code']
source_index = []
for col in target_embedding:
    source_index += [i for (i,col) in enumerate(df.columns) if col== 'source_id_hash_code']

target_id_index = []
for col in target_embedding:
    target_id_index += [i for (i,col) in enumerate(df.columns) if col== 'target_id_hash_code']

# Now start_with_cat_columns is a dictionary that maps each item in user_categorical_columns
# to a list of tuples where each tuple is (index, column_name)
#%%
syndicator_col = [col  for col in df.columns if 'syndi' in col if col != 'syndicator_code']

#%%
placement_id_col = [col  for col in df.columns if 'placement_id' in col if col != 'placement_id_code']
#%%
region_col = [col  for col in df.columns if 'region_' in col if col != 'region_code']

#%%
item_oh_cat_columns = [col for col in item_oh_cat_columns if col not in syndicator_col + placement_id_col]

#%%
user_oh_cat_columns = [col for col in user_oh_cat_columns if col not in region_col]
ds = FMds2(df)

#%%
#del df
#%%
train_size = int(0.8 * len(ds))  # 80% for training
val_size = len(ds) - train_size  # Remaining 20% for validation
train_dataset, val_dataset = random_split(ds, [train_size, val_size])
del ds
#%%
#%%
model = RCM()
#model = RCM.load_from_checkpoint('C:/Users/amityu/DataspellProjects/recommender/checkpoints128/rcm-epoch=14-val_loss=0.47.ckpt')
trainer = pl.Trainer(num_sanity_val_steps=0, max_epochs=150,fast_dev_run=False, callbacks=[pl.callbacks.ModelCheckpoint
                                                                (monitor='val_auc',dirpath='/content/drive/MyDrive/reco2/cp2',filename='FM-{epoch:02d}-{val_loss:.2f}',save_top_k=3,mode='min')])
trainer.fit(model, train_dataloaders=DataLoader(train_dataset, batch_size=512, num_workers=4), val_dataloaders=DataLoader(val_dataset, batch_size=512, num_workers=4))#,ckpt_path='C:/Users/amityu/DataspellProjects/recommender/checkpoints128/rcm-epoch=09-val_loss=0.48.ckpt')
print('done')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name           | Type        | Params
------------------------------------------------
0  | placement_id   | Embedding   | 9.1 K 
1  | syndicator     | Embedding   | 16.1 K
2  | region         | Embedding   | 7.1 K 
3  | target_id      | Embedding   | 276 K 
4  | source         | Embedding   | 400 K 
5  | campaign       | Embedding   | 116 K 
6  | fc_user        | Linear      | 21.6 K
7  | fc_user_full   | Linear      | 4.7 K 
8  | fc_item        | Linear      | 4.7 K 
9  | fc_item_full   | Linear      | 6.7 K 
10 | deep           | Linear      | 16.5 K
11 | validation_auc | BinaryAUROC | 0     
12 | output         | Lin

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
df.head()


NameError: name 'df' is not defined